In [14]:
from pathlib import Path
import os

# Force change into the fine-tune repo root
target = Path("Fine-tune project") / "mental-health-fine-tuned-llm-SA"
if not target.exists():
    # fallback for OneDrive full path (if above relative path fails)
    target = Path(r"C:\Users\User\OneDrive\Desktop\(ZA4309) Emerging Technologies in Intelligence\Project\Fine-tune project\mental-health-fine-tuned-llm-SA")

os.chdir(target)
print("CWD now:", Path.cwd())
print("Has src/?", (Path("src")).exists())
print("Has runs/?", (Path("runs")).exists())


CWD now: c:\Users\User\OneDrive\Desktop\(ZA4309) Emerging Technologies in Intelligence\Project\Fine-tune project\mental-health-fine-tuned-llm-SA
Has src/? True
Has runs/? True


In [15]:
from pathlib import Path

BASE_MODEL = "Qwen/Qwen2.5-0.5B-Instruct"
OUTPUT_DIR = Path("runs") / "sft-qwen"          # where Trainer saved
CKPT_DIR   = OUTPUT_DIR / "checkpoint-last"     # created by Option A
VAL_FILE   = Path("data/processed/counselchat_val.jsonl")

# how many val examples to score (keep small on CPU first)
N_EVAL = 100    # you can raise later if it's fast enough

assert VAL_FILE.exists(), f"Missing: {VAL_FILE}"
assert OUTPUT_DIR.exists(), f"Missing: {OUTPUT_DIR}"
print("Eval file:", VAL_FILE)
print("Checkpoint dir (expected):", CKPT_DIR)


Eval file: data\processed\counselchat_val.jsonl
Checkpoint dir (expected): runs\sft-qwen\checkpoint-last


In [17]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# tokenizer from the base model (safe), weights from local checkpoint
tok = AutoTokenizer.from_pretrained(BASE_MODEL)

ckpt_to_load = CKPT_DIR if CKPT_DIR.exists() else OUTPUT_DIR
print("Loading from:", ckpt_to_load)

model = AutoModelForCausalLM.from_pretrained(
    ckpt_to_load.as_posix(),
    torch_dtype=torch.float32,        # CPU
    local_files_only=True             # don't hit the hub
).to("cpu")

tok.padding_side = "right"
model.config.use_cache = False

print("Loaded OK.")


Loading from: runs\sft-qwen


ValueError: Unrecognized model in runs/sft-qwen. Should have a `model_type` key in its config.json, or contain one of the following strings in its name: aimv2, aimv2_vision_model, albert, align, altclip, apertus, arcee, aria, aria_text, audio-spectrogram-transformer, autoformer, aya_vision, bamba, bark, bart, beit, bert, bert-generation, big_bird, bigbird_pegasus, biogpt, bit, bitnet, blenderbot, blenderbot-small, blip, blip-2, blip_2_qformer, bloom, blt, bridgetower, bros, camembert, canine, chameleon, chinese_clip, chinese_clip_vision_model, clap, clip, clip_text_model, clip_vision_model, clipseg, clvp, code_llama, codegen, cohere, cohere2, cohere2_vision, colpali, colqwen2, conditional_detr, convbert, convnext, convnextv2, cpmant, csm, ctrl, cvt, d_fine, dab-detr, dac, data2vec-audio, data2vec-text, data2vec-vision, dbrx, deberta, deberta-v2, decision_transformer, deepseek_v2, deepseek_v3, deepseek_vl, deepseek_vl_hybrid, deformable_detr, deit, depth_anything, depth_pro, deta, detr,

In [ ]:
from pathlib import Path

val_path = Path("data/processed/counselchat_val.jsonl")
preds, refs = [], []

for line in open(val_path, "r", encoding="utf-8"):
    ex = json.loads(line)
    prompt = ex["input"]
    ref = ex["output"]
    ids = tok(prompt, return_tensors="pt", truncation=True, max_length=512).to("cpu")
    out = model.generate(**ids, max_new_tokens=256, do_sample=False)
    text = tok.decode(out[0], skip_special_tokens=True)
    # simple post-cut: take only the text after the prompt if it appears
    pred = text[len(prompt):].strip() if text.startswith(prompt) else text.strip()
    preds.append(pred)
    refs.append(ref)

print(len(preds), "predictions")


In [ ]:
from datasets import load_metric
# ROUGE & BLEU
from rouge_score import rouge_scorer
import numpy as np

# ROUGE-L
scorer = rouge_scorer.RougeScorer(["rougeLsum"], use_stemmer=True)
rouges = [scorer.score(r, p)["rougeLsum"].fmeasure for r,p in zip(refs, preds)]
print("ROUGE-L (mean):", float(np.mean(rouges)))

# SacreBLEU
from sacrebleu import corpus_bleu
bleu = corpus_bleu(preds, [refs])
print("SacreBLEU:", bleu.score)
